NOTE: DEZE CODE IS NOG NIET VOLLEDIG. We willen twee hoeken ($\theta$ en $\varphi$) meten met behulp van een tweetal microfoons op dezelfde rij ($\theta$) een een tweetal op dezelfde kolom ($\varphi$), terwijl deze code je maar één hoek geeft op basis van één paar. De code voor beide hoeken is identiek, maar er moeten nog kleine aanpassingen worden gemaakt in het tweede deel van de code.

De code bestaat uit twee delen. Het eerste deel doet de meting en alle setup. Het tweede deel berekend de hoeken. Onderscheid tussen de delen is gemaakt zodat we niet een aparte meting hoeven te doen per hoek. We hebben op deze manier genoeg aan één meting.

DEEL 1: Run onderstaande code om een meting te doen. Pas de geluidssnelheid (v) en de afstanden tussen de microfoons (d) onderin dit code-blok aan zoals gewenst. 

In [2]:
#>>>>FINDING AUDIO DEVICE
import sounddevice as sd

indev = sd.query_devices("micArray16")
print(f"Found input device {indev}")

#>>>>>SETTINGS
freq = 44000 #frequency
nchan = 16 #number of channels
sd.default.samplerate = freq
sd.default.channels = nchan
sd.default.device = indev['index']

#>>>>>RECORD
import numpy as np

npt = 220000 #total number of measurements per channel (TIME * FREQ)
dur = npt/freq

print(">>> Start recording for %5.2fs (%d samples per channel)" % (dur,npt))
rec = sd.rec(npt)
sd.wait()
untransposed = np.array(rec)
arec = np.transpose(untransposed) #<<-- THIS IS OUR DATA
print(">>> Done (%s x %s samples)" % (str(np.shape(arec)[0]), str(np.shape(arec)[1])))

#CALCULATING THE TIME DIFFERENCE
def time_difference(mic1index, mic2index, arec, freq): #<<-- use 1-index
    #difference in datapoints
    maxmic1 = max(arec[mic1index])
    dp1 = arec[mic1index].index(maxmic1)
    maxmic2 = max(arec[mic2index])
    dp2 = arec[mic2index].index(maxmic2)
    dpd = dp1 - dp2 #<<-- NOTE1 THE SIGNIFICANE OF THE SIGN!

    #time difference
    dtime = dpd/freq
    return dtime

#CALCULATING THE ANGLE
import math 

def angle(mic1index, mic2index, arec, freq):
    v = 1 #speed of sound
    d = 1 #distance between mics
    theta = math.acos((v * time_difference(mic1index, mic2index, arec, freq))/d)
    return theta


Hier komt code


DEEL 2: Pas "EERSTE_MIC" en "TWEEDE_MIC" aan. Merk op dat de volgorde uitmaakt voor de interpretatie van de hoek. Lees hiervoor de comment onderin de code.

In [ ]:
EERSTE_MIC = 1 #in 1-index
TWEEDE_MIC = 2 #in 1-index

print("Deze hoek is ", angle(EERSTE_MIC, TWEEDE_MIC, arec, freq), " radialen.")

#>>>>>>NOTEEEEEE: An angle theta should be interpreted as follows:
#Draw a line perpendicular to the surface of the microphone-array, through mic1. Choose the line-segment that is on the source-side of the surface. Now rotate the line-segment around mic1, towards mic2 around an angle of theta radians. The line segment will point towards the source.